#  VisionGuard AI – Model Training

---

##  Notebook Overview  

In this notebook, I will:  

-  Define a **PyTorch CNN model** for defect detection.  
-  Set **loss function & optimizer**.  
-  Train the model using the screw dataset.  
-  Save the trained model for later evaluation & deployment.  


##  Import Libraries  




In [1]:
import os, copy, time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report



##  Define Configurations
I will define dataset paths, batch size, learning rate, and preprocessing transforms.


In [6]:

# config 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

dataset_dir = r"E:\Final project of Ats\screw_dataset"
batch_size  = 32
num_classes = 6
epochs      = 30
lr          = 1e-4

# transforms
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])
val_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

# datasets + dataloaders
train_dataset = datasets.ImageFolder(os.path.join(dataset_dir, "train"), transform=train_tfms)
val_dataset   = datasets.ImageFolder(os.path.join(dataset_dir, "val"), transform=val_tfms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)


Using device: cpu
Classes: ['good', 'manipulated_front', 'scratch_head', 'scratch_neck', 'thread_side', 'thread_top']


##  Load Pretrained ResNet50
I will use ResNet50 pretrained on ImageNet, replace the final fully connected layer, and prepare it for training.


In [8]:

# instead of weights=models.ResNet50_Weights.IMAGENET1K_V2
model = models.resnet50(pretrained=True)

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


d:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\A.C/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%


## Training & Validation Loop
I will train the model, evaluate after each epoch, and save the best model weights.


In [9]:
best_acc = 0.0
best_wts = copy.deepcopy(model.state_dict())

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # ---- train ----
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        preds = outputs.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    train_loss = running_loss / total
    train_acc = correct / total

    # ---- validation ----
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            val_loss += loss.item() * x.size(0)
            preds = outputs.argmax(1)
            val_correct += (preds == y).sum().item()
            val_total += y.size(0)
    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Train Loss {train_loss:.4f} Acc {train_acc:.4f} | "
          f"Val Loss {val_loss:.4f} Acc {val_acc:.4f}")

    # save checkpoint
    if val_acc > best_acc:
        best_acc = val_acc
        best_wts = copy.deepcopy(model.state_dict())
        print(">> Saved new best model!")

# load best weights
model.load_state_dict(best_wts)



Epoch 1/30
Train Loss 1.2195 Acc 0.6208 | Val Loss 0.9797 Acc 0.7143
>> Saved new best model!

Epoch 2/30
Train Loss 0.9680 Acc 0.6785 | Val Loss 1.0530 Acc 0.7500
>> Saved new best model!

Epoch 3/30
Train Loss 0.8711 Acc 0.7184 | Val Loss 0.7534 Acc 0.7679
>> Saved new best model!

Epoch 4/30
Train Loss 0.6865 Acc 0.7694 | Val Loss 0.8813 Acc 0.6429

Epoch 5/30
Train Loss 0.6730 Acc 0.7805 | Val Loss 0.9720 Acc 0.7679

Epoch 6/30
Train Loss 0.6045 Acc 0.8115 | Val Loss 0.7730 Acc 0.7143

Epoch 7/30
Train Loss 0.5005 Acc 0.8337 | Val Loss 0.8937 Acc 0.6964

Epoch 8/30
Train Loss 0.4625 Acc 0.8448 | Val Loss 1.7169 Acc 0.4643

Epoch 9/30
Train Loss 0.4259 Acc 0.8470 | Val Loss 0.8079 Acc 0.7500

Epoch 10/30
Train Loss 0.3511 Acc 0.8847 | Val Loss 0.5468 Acc 0.8036
>> Saved new best model!

Epoch 11/30
Train Loss 0.2382 Acc 0.9180 | Val Loss 0.4120 Acc 0.8750
>> Saved new best model!

Epoch 12/30
Train Loss 0.3074 Acc 0.8980 | Val Loss 1.0231 Acc 0.6786

Epoch 13/30
Train Loss 0.3471 A

<All keys matched successfully>

# Best Model

In [ ]:
torch.save(best_wts, "best_model.pth")


## Step 4: Evaluation on Validation Set


In [14]:
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        preds = outputs.argmax(1)
        y_true.extend(y.cpu().tolist())
        y_pred.extend(preds.cpu().tolist())

print("\nClassification Report of Validation accuracy:")
print(classification_report(y_true, y_pred, target_names=class_names))



Classification Report of Validation accuracy:
                   precision    recall  f1-score   support

             good       0.95      1.00      0.97        36
manipulated_front       1.00      1.00      1.00         4
     scratch_head       0.67      1.00      0.80         4
     scratch_neck       1.00      0.25      0.40         4
      thread_side       0.75      0.75      0.75         4
       thread_top       0.67      0.50      0.57         4

         accuracy                           0.89        56
        macro avg       0.84      0.75      0.75        56
     weighted avg       0.90      0.89      0.88        56



## Save Model into `model.py`



In [11]:
export_path = "model.py"
with open(export_path, "w") as f:
    f.write(f"""import torch
import torch.nn as nn
from torchvision import models
            
def get_trained_model(num_classes={num_classes}, device=None):
    model = models.resnet50()
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    state_dict = {best_wts}
    model.load_state_dict(state_dict)
    if device:
        model = model.to(device)
    model.eval()
    return model
""")
print(f" Trained model exported to {export_path}")


 Trained model exported to model.py
